Set this variable to your osgeopy-data directory so that the following
examples will work without editing. 
* We'll use the os.path.join() function to combine this directory and the filenames to make a complete path. Of
course, you can type the full path to the file for each example if you'd prefer.

In [1]:
import os
import sys
data_dir = r'osgeopy-data'
# data_dir =

## Introduction

In [2]:
# Import the module.
from osgeo import ogr

# Get the GeoJSON driver.
driver = ogr.GetDriverByName('GeoJSON')
print(driver)

# It's not case sensitive, so this also works.
driver = ogr.GetDriverByName('geojson')
print(driver)

# This does not work because the real name is 'Esri shapefile'.
driver = ogr.GetDriverByName('shapefile')
print(driver)

<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x7f7788600e10> >
<osgeo.ogr.Driver; proxy of <Swig Object of type 'OGRDriverShadow *' at 0x7f7788600f00> >
None


In [3]:
# Print out a list of drivers.
import osgeopy as ospy
ospy.print_drivers()

ESRIC (readonly)
FITS (read/write)
PCIDSK (read/write)
netCDF (read/write)
PDS4 (read/write)
VICAR (read/write)
JP2OpenJPEG (readonly)
PDF (read/write)
MBTiles (read/write)
BAG (read/write)
EEDA (readonly)
OGCAPI (readonly)
ESRI Shapefile (read/write)
MapInfo File (read/write)
UK .NTF (readonly)
LVBAG (readonly)
OGR_SDTS (readonly)
S57 (read/write)
DGN (read/write)
OGR_VRT (readonly)
REC (readonly)
Memory (read/write)
CSV (read/write)
NAS (readonly)
GML (read/write)
GPX (read/write)
LIBKML (read/write)
KML (read/write)
GeoJSON (read/write)
GeoJSONSeq (read/write)
ESRIJSON (readonly)
TopoJSON (readonly)
Interlis 1 (read/write)
Interlis 2 (read/write)
OGR_GMT (read/write)
GPKG (read/write)
SQLite (read/write)
OGR_DODS (readonly)
WAsP (read/write)
PostgreSQL (read/write)
OpenFileGDB (readonly)
DXF (read/write)
CAD (readonly)
FlatGeobuf (read/write)
Geoconcept (read/write)
GeoRSS (read/write)
GPSTrackMaker (read/write)
VFK (readonly)
PGDUMP (read/write)
OSM (readonly)
GPSBabel (read/write)

## Reading vector data 

### Accessing specific features

In [4]:
# Open the data source for the examples.
fn = os.path.join(data_dir, 'global', 'ne_50m_populated_places.shp')
ds = ogr.Open(fn, 0)
if ds is None:
    sys.exit('Could not open {0}.'.format(fn))
lyr = ds.GetLayer(0)

# Get the total number of features and the last one.
num_features = lyr.GetFeatureCount()
last_feature = lyr.GetFeature(num_features - 1)
print(last_feature.NAME)

Hong Kong


#### Test what happens if you try to loop through a layer twice.

In [5]:
#The second loop should not print anything.
fn = os.path.join(data_dir, 'washington', 'large_cities.geojson')
#print(fn)
ds = ogr.Open(fn, 0)
lyr = ds.GetLayer(0)
print('First loop')
for feat in lyr:
    print(feat.GetField('Name'), feat.GetField('Population'))
print('Second loop')
for feat in lyr:
    pt = feat.geometry()
    print(feat.GetField('Name'), pt.GetX(), pt.GetY())

# # But it will if you reset reading first.
ds = ogr.Open(fn, 0)
lyr = ds.GetLayer(0)
print('First loop')
for feat in lyr:
    print(feat.GetField('Name'), feat.GetField('Population'))
print('Second loop')
lyr.ResetReading() # This is the important line.
for feat in lyr:
    pt = feat.geometry()
    print(feat.GetField('Name'), pt.GetX(), pt.GetY())

First loop
Bellingham 67171
Everett 91488
Shoreline 53025
Seattle 563374
Bellevue 109569
Renton 50052
Kent 79524
Federal Way 83259
Tacoma 193556
Spokane 195629
Yakima 71845
Vancouver 143560
Kennewick 54693
Second loop
Bellingham -122.48822021484375 48.759552001953125
Everett -122.20207214355469 47.97898864746094
Shoreline -122.34152221679688 47.75564956665039
Seattle -122.33206939697266 47.6062126159668
Bellevue -122.20067596435547 47.610382080078125
Renton -122.2170639038086 47.482879638671875
Kent -122.2348403930664 47.38093185424805
Federal Way -122.31262969970703 47.32231903076172
Tacoma -122.44429779052734 47.25288009643555
Spokane -117.4260482788086 47.658782958984375
Yakima -120.50589752197266 46.60206604003906
Vancouver -122.66148376464844 45.638729095458984
Kennewick -119.1372299194336 46.211246490478516
First loop
Bellingham 67171
Everett 91488
Shoreline 53025
Seattle 563374
Bellevue 109569
Renton 50052
Kent 79524
Federal Way 83259
Tacoma 193556
Spokane 195629
Yakima 71845
Va

### Extract certain features from a shapefile and save them to another file.

In [6]:
import sys
from osgeo import ogr

# Open the folder data source for writing
ds = ogr.Open(r'osgeopy-data/global', 1)
if ds is None:
    sys.exit('Could not open folder.')

# Get the input shapefile
in_lyr = ds.GetLayer('ne_50m_populated_places')

# Create a point layer
if ds.GetLayer('capital_cities'):
    ds.DeleteLayer('capital_cities')
out_lyr = ds.CreateLayer('capital_cities',
                         in_lyr.GetSpatialRef(),
                         ogr.wkbPoint)
out_lyr.CreateFields(in_lyr.schema)

# Create a blank feature
out_defn = out_lyr.GetLayerDefn()
out_feat = ogr.Feature(out_defn)

for in_feat in in_lyr:
    if in_feat.GetField('FEATURECLA') == 'Admin-0 capital':

        # Copy geometry and attributes
        geom = in_feat.geometry()
        out_feat.SetGeometry(geom)
        for i in range(in_feat.GetFieldCount()):
            value = in_feat.GetField(i)
            out_feat.SetField(i, value)

        # Insert the feature
        out_lyr.CreateFeature(out_feat)

# Close files
del ds

### Viewing your data 

In [7]:
# Print name and population attributes.
import osgeopy as ospy

fn = os.path.join(data_dir, 'global', 'ne_50m_populated_places.shp')
ospy.print_attributes(fn, 3, ['NAME', 'POP_MAX'])



FID    Geometry                  NAME           POP_MAX    
0      POINT (32.533, 0.583)     Bombo          75000      
1      POINT (30.275, 0.671)     Fort Portal    42670      
2      POINT (15.799, 40.642)    Potenza        69060      
3 of 1249 features


## Getting metadata

In [10]:
# Open the large_cities data source.
fn = os.path.join(data_dir, 'washington', 'large_cities.geojson')
ds = ogr.Open(fn)
if ds is None:
    sys.exit('Could not open {0}.'.format(fn))

# Get the spatial extent.
lyr = ds.GetLayer(0)
extent = lyr.GetExtent()
print(extent)
print('Upper left corner: {}, {}'.format(extent[0], extent[3]))
print('Lower right corner: {}, {}'.format(extent[1], extent[2]))

# Get geometry type
print(lyr.GetGeomType())
print(lyr.GetGeomType() == ogr.wkbPoint)
print(lyr.GetGeomType() == ogr.wkbPolygon)

# Get geometry type as human-readable string.
feat = lyr.GetFeature(0)
print(feat.geometry().GetGeometryName())

# Get spatial reference system. The output is also in listing3_2.py.
print(lyr.GetSpatialRef())

# Get field names and types
for field in lyr.schema:
    print(field.name, field.GetTypeName())

(-122.66148376464844, -117.4260482788086, 45.638729095458984, 48.759552001953125)
Upper left corner: -122.66148376464844, 48.759552001953125
Lower right corner: -117.4260482788086, 45.638729095458984
1
True
False
POINT
GEOGCS["NAD83",
    DATUM["North_American_Datum_1983",
        SPHEROID["GRS 1980",6378137,298.257222101,
            AUTHORITY["EPSG","7019"]],
        AUTHORITY["EPSG","6269"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4269"]]
CITIESX020 Integer
FEATURE String
NAME String
POP_RANGE String
POPULATION Integer
FIPS55 String
COUNTY String
FIPS String
STATE String
STATE_FIPS String
DISPLAY Integer


## Using OGR exceptions

In [12]:
# Try running this when output/africa.geojson already exists in order to raise
# the error.

# Turn on OGR exceptions. Try commenting this out to see how the behavior
# changes.
ogr.UseExceptions()

fn = os.path.join(data_dir, 'output', 'africa.geojson')
driver = ogr.GetDriverByName('GeoJSON')
print('Doing some preliminary analysis...')

try:
    # This will fail if the file already exists
    ds = driver.CreateDataSource(fn)
    lyr = ds.CreateLayer('layer')
    # Do more stuff, like fields and save data
except RuntimeError as e:
    # This runs if the data source already exists and an error was raised
    print(e)

print('Doing some more analysis...')

Doing some preliminary analysis...
Failed to create GeoJSON datasource: osgeopy-data/output/africa.geojson: osgeopy-data/output/africa.geojson: No such file or directory
Doing some more analysis...
